In [0]:
#Check GPU Runtime
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

In [0]:
#Pre-requisite: If the code is executed in Google Colab, download the dataset and upload it to your google drive in the same location
#Dataset: https://drive.google.com/file/d/19kPdA5v7gtAMU5QbIxHxjWj_HdiYPcZf/view?usp=sharing
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

In [0]:
#Required to run after every session timeout
pip install librosa --upgrade

In [4]:
## Implementation of DCGAN - as described by Radford et al. 2015
#1. Importing required packages
import numpy as np
import matplotlib.pyplot as plt
import librosa
from librosa import display
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Reshape
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import LeakyReLU, Dropout
from keras.layers import BatchNormalization
from keras.optimizers import Adam, RMSprop


#2. Constructing MusicDCGAN class that contains the model for the discriminator and generator
class MusicDCGAN(object):
    def __init__(self, spectogram_time=640, spectogram_frequency=128, channel=1):
        self.img_rows = spectogram_frequency
        self.img_cols = spectogram_time
        self.channel = channel
        self.D = None   # discriminator
        self.G = None   # generator
        self.AM = None  # adversarial model
        self.DM = None  # discriminator model

    #3. Setting up discriminator model
    def discriminator(self):
        if self.D:
            return self.D
        self.D = Sequential()
        depth = 128
        dropout = 0.4
        input_shape = (self.img_rows, self.img_cols, self.channel)
        self.D.add(Conv2D(depth*1, 5, strides=2, input_shape=input_shape,\
            padding='same'))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        self.D.add(Conv2D(depth*2, 5, strides=2, padding='same'))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        self.D.add(Conv2D(depth*4, 5, strides=2, padding='same'))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        self.D.add(Conv2D(depth*8, 5, strides=2, padding='same'))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        self.D.add(Conv2D(depth*16, 5, strides=2, padding='same'))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        self.D.add(Flatten())
        self.D.add(Dense(1))
        self.D.add(Activation('sigmoid'))
        print("Discrimnator Summary")
        self.D.summary()
        return self.D

    #4. Setting up generator model
    def generator(self):
        if self.G:
            return self.G
        self.G = Sequential()
        dropout = 0.4
        depth = 1024
        dim = 8
        dim2 = 40
        self.G.add(Dense(dim*dim2*depth, input_dim=64))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))
        self.G.add(Reshape((dim, dim2, depth)))
        self.G.add(Dropout(dropout))
        self.G.add(UpSampling2D())
        self.G.add(Conv2DTranspose(int(depth/2), 5, padding='same'))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))
        self.G.add(UpSampling2D())
        
        self.G.add(Conv2DTranspose(int(depth/4), 5, padding='same'))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))
        self.G.add(UpSampling2D())
        
        self.G.add(Conv2DTranspose(int(depth/8), 5, padding='same'))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))
        self.G.add(UpSampling2D())
        
        #self.G.add(Conv2DTranspose(int(depth/16), 5, padding='same'))
        #self.G.add(BatchNormalization(momentum=0.9))
        #self.G.add(Activation('relu'))

        self.G.add(Conv2DTranspose(1, 5, padding='same'))
        self.G.add(Activation('tanh'))
        print("Generator Summary")
        self.G.summary()
        return self.G

    #5. Defining discriminator model
    def discriminator_model(self):
        if self.DM:
            return self.DM
        optimizer = RMSprop(lr=0.0002, decay=6e-8)
        self.DM = Sequential()
        self.DM.add(self.discriminator())
        self.DM.compile(loss='binary_crossentropy', optimizer=optimizer,\
            metrics=['accuracy'])
        return self.DM

    #6. Defining generator model
    def adversarial_model(self):
        if self.AM:
            return self.AM
        optimizer = RMSprop(lr=0.0001, decay=3e-8)
        self.AM = Sequential()
        self.AM.add(self.generator())
        self.AM.add(self.discriminator())
        self.AM.compile(loss='binary_crossentropy', optimizer=optimizer,\
            metrics=['accuracy'])
        return self.AM

#7. Constructing TrainDCGAN class that contains the training algorithm for the models
class TrainDCGAN(object):
    def __init__(self):
        self.img_rows = 128
        self.img_cols = 640
        #https://drive.google.com/file/d/1vKVt7sd2f8D5aSNxoggcdHbrThHxlXx7/view?usp=sharing - Download file and upload it to Google drive in the same path.
        train_data = np.load('/content/drive/My Drive/DL/GAN/shuffled_train.npz')
        self.x_train_rot = train_data['arr_0']
        size = 999
        self.x_train = np.empty((size,self.img_rows,self.img_cols))
        for i in range(size):
            self.x_train[i,:,:] = np.rot90(self.x_train_rot[i,:,:],3)
        print(self.x_train.shape) 
        self.x_train = self.x_train.reshape(-1, self.img_rows,self.img_cols, 1).astype(np.float32)


        #8. Creating objects for MusicDCGAN and calling the models 
        self.MusicDCGAN = MusicDCGAN()
        self.discriminator =  self.MusicDCGAN.discriminator_model()
        self.adversarial = self.MusicDCGAN.adversarial_model()
        self.generator = self.MusicDCGAN.generator()
        self.disc_loss = []
        self.gen_loss = []

    #9. Defining the training parameters
    def train(self, epochs, batch_size, save_interval):
        noise_input = None
        if save_interval>=0:
            noise_input = np.random.uniform(-1.0, 1.0, size=[batch_size, 64])
        for i in range(epochs):
            #print("Input sample data shape and spectrogram")
            #print(self.x_train.shape)
            #librosa.display.specshow((self.x_train[np.random.randint(0, 999),:,:,0]), y_axis='mel', fmax=8000, x_axis='time')
            #plt.colorbar(format='%+2.0f dB')
            #plt.show()    
            images_train = self.x_train[np.random.randint(0, self.x_train.shape[0], size=batch_size), :, :, :]
            #print("Input training data shape and spectrogram")
            #print(images_train.shape)
            #librosa.display.specshow(images_train[0,:,:,0], y_axis='mel', fmax=8000, x_axis='time')
            #plt.colorbar(format='%+2.0f dB')
            #plt.show()            
            noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 64])
            images_fake = self.generator.predict(noise)
            x = np.concatenate((images_train, images_fake))
            y = np.ones([2*batch_size, 1])
            y[batch_size:, :] = 0
            d_loss = self.discriminator.train_on_batch(x, y)
            y = np.ones([batch_size, 1])
            noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 64])
            a_loss = self.adversarial.train_on_batch(noise, y)
            log_mesg = "%d: [D loss: %f, acc: %f]" % (i, d_loss[0], d_loss[1])
            log_mesg = "%s  [A loss: %f, acc: %f]" % (log_mesg, a_loss[0], a_loss[1])
            self.disc_loss.append(d_loss[0])
            self.gen_loss.append(a_loss[0])
            print(log_mesg)
            if (i+1)%save_interval==0:
              self.generate_spectrogram(noise_input, i)


      #10. Generating new spectrogram at given sample interval
    def generate_spectrogram(self, noise, epoch):
        predictions = self.generator.predict(noise)
        print("Predicted data shape")
        print(predictions.shape)
        print("Reshaping Output data")
        generated_seq = np.reshape(predictions[0], (128,640))
        print(generated_seq.shape)
        self.convert_to_audio(generated_seq, epoch)
        librosa.display.specshow(generated_seq, y_axis='mel', fmax=8000, x_axis='time')
        plt.colorbar(format='%+2.0f dB')
        plt.savefig("/content/drive/My Drive/DL/GAN/Results/DCGAN_MUSIC/GeneratedOutputForEpoch-%d.png" % epoch)
        plt.show()
        plt.close()
      
      #11. Generate audio sample from spectogram
    def convert_to_audio (self, dbspectogram, epoch):
        # db_to_power(S_db) ~= ref * 10.0**(S_db / 10)
        audio_spectogram = librosa.core.db_to_power(dbspectogram)
        #Converting spectogram into time series array
        audio_timeSeries = librosa.feature.inverse.mel_to_audio(audio_spectogram, sr = 22050, hop_length=1024)
        #Converting time series array into audio file
        audiofile_path = "/content/drive/My Drive/DL/GAN/Results/DCGAN_MUSIC/GeneratedMusicForEpoch-%d.wav" % epoch
        librosa.output.write_wav(audiofile_path, audio_timeSeries, sr=22050, norm=False)




Using TensorFlow backend.


In [0]:
mus_dcgan = TrainDCGAN()
mus_dcgan.train(epochs=10000, batch_size=32, save_interval=100)

372: [D loss: 24.737602, acc: 0.500000]  [A loss: 2497.455566, acc: 0.000000]
